In [ ]:
"""
Imports
"""
# import dataset
from classes.loaddata import OurDataset
# import np
import numpy as np
# import torch
import torch
import torchvision.transforms as transforms
from torchvision.utils import save_image
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim
import torchvision
# import pickle
import pickle
# image processer
from PIL import Image

In [ ]:
"""
Convolutional Neural Network
"""
class CNN(nn.Module):
    def __init__(self, in_channel = 3, num_classes = 200):
        super(CNN, self).__init__()
        
        # layers here

    def forward(self, x):
        pass

In [ ]:
"""
Set hyperparameters
"""
# Hyperparameters
input_size = 32
num_classes = 200
learning_rate = 0.01
batch_size = 32
num_epochs = 5

In [ ]:
"""
Image transformations here:
transformations that are necessary: 
- transforms.lambda since some have 1 input channel instead of 3
- to tensor as a format
"""
my_transforms = transforms.Compose([
    transforms.Lambda(lambda img: Image.fromarray(img) if isinstance(img, np.ndarray) else img),  
    transforms.Resize((32, 32)),  
    transforms.Lambda(lambda img: img.convert('RGB')),  
    transforms.ToTensor()  
])

In [ ]:
"""
If we have an Nvidia GPU, use it!
"""
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
"""
Loading in all data using the dataset class in loaddata.py
"""
# initialize dataset
train_dataset = OurDataset(csv_file = "datafile/train_images.csv", root_dir = "datafile/train_images", transform = my_transforms)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True)

test_dataset = OurDataset(csv_file = "datafile/train_images.csv", root_dir = "datafile/test_images", transform = my_transforms)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, shuffle = True, num_workers=4)

class_dictionary = np.load("datafile/class_names.npy", allow_pickle=True).item()
class_names = list(class_dictionary.keys())
class_names = [name.split('.',1)[1] for name in class_names]

In [ ]:
"""
Instantiate model
"""
model = CNN().to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
"""
Checking accuracy
*Note: only working for traindata only so far
"""
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    # dont use gradient to check cause we dont need it here
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            y = y - 1
            scores = model(x)
            # find index for second dimension?
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum().item()
            num_samples += predictions.size(0)
        
        print(f"got {num_correct} / {num_samples} with accuracy {(float(num_correct)/float(num_samples))*100:.2f}")

    model.train()

In [ ]:
"""
Run

todo: save model

*warning: long runtimes
"""
# Train network
for epoch in range(num_epochs):
    print(f"epoch nr. {epoch}")
    # data: image, target: label for each image
    for batch_index, (data, targets) in enumerate(train_loader):
        # to cuda if possible else cpu
        data = data.to(device=device)
        targets=targets.to(device=device)

        # print(data.shape)

        # forward
        scores = model(data)
        loss = loss_function(scores, targets - 1)

        # backward
        # set to zero from previous forward props
        optimizer.zero_grad()
        loss.backward()

        # gradient descent
        optimizer.step()

    # check accuracy
    check_accuracy(train_loader, model)
    # check_accuracy(test_loader, model)